In [ ]:
import csv
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import networkx as nx
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from folium.plugins import HeatMap
import genderize
from genderize import Genderize
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
import gzip
from gensim import corpora, models, similarities
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import os
import sys
import logging
import argparse
import lxml
from lxml import html
import requests
import bs4
import re
import zipfile
import io
import tensorflow as tf
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
import collections

**Geographic Map of Organizations Receiving Grants from NIH**

In [ ]:
data = pd.read_csv("organization.csv", names=['organization_name', 'latitude', 'longitude'])
m = folium.Map(location=[20,0], tiles='Mapbox Bright', zoom_start=2)
for i in range(len(data)):
    long = float(data.iloc[i]['longitude'])
    lat = float(data.iloc[i]['latitude'])
    popup = folium.Popup(data.iloc[i]['organization_name'], parse_html=True) 
    folium.map.Marker([lat, long], popup).add_to(m)
# m.save('organization.html')
info = pd.read_csv("rganization_information.csv", names=['organization_name', 'latitude', 'longitude', 'number_pis', 'total_grant_amount'])
if info['total_grant_amount'] <= 100000:
     
#Number of Grants - size
#Amount of Grant - color
#Number of PI involved per Grant - popup

**Heat Map Distribution of Active NIH Grants**

In [ ]:
data = pd.read_csv('genderizer.csv', names=['full_name', 'first_name', 'gender', 'latitude', 'longitude'])
n = folium.Map([20, 0], tiles= 'Mapbox Bright', zoom_start=2)
lat = data['latitude'].astype(float)
long = data['longitude'].astype(float)
heat_dat = data[['latitude', 'longitude']]
heat_dat = heat_dat.dropna(axis=0, subset=['latitude','longitude'])
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_dat.iterrows()]
HeatMap(heat_data).add_to(n)
n.save("heatmapnih.html")

**Genderizer - Heat Map Distribution of Female PIs regarding Active NIH Grants**

In [ ]:
data = pd.read_csv('genderizerfemale.csv', names=['full_name', 'first_name', 'gender', 'latitude', 'longitude', 'organization'])
g = folium.Map([20, 0], tiles= 'Mapbox Bright', zoom_start=2)
lat = data['latitude'].astype(float)
long = data['longitude'].astype(float)
heat_dat = data[['latitude', 'longitude']]
heat_dat = heat_dat.dropna(axis=0, subset=['latitude','longitude'])
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_dat.iterrows()]
HeatMap(heat_data, gradient={.4: 'red', .65: 'green', 1: 'blue'}).add_to(g)
g.save("gheatmapnih.html")

**Gensim**

In [1]:
# http://projector.tensorflow.org/ with 'abstractnih.csv' uploaded as metadata

In [ ]:
csv.field_size_limit(1000000000)
dictionary = corpora.Dictionary(line.lower().split() for line in open('abstract.csv'))
print(dictionary)
class MyCorpus(object):
    def __iter__(self):
        for line in open('abstract.csv'):
            yield dictionary.doc2bow(line.lower().split())
corpus_memory_friendly = MyCorpus() 
print(corpus_memory_friendly)
for vector in corpus_memory_friendly:  
    print(vector)
tfidf = models.TfidfModel(corpus_memory_friendly)
corpus_tfidf = tfidf[corpus_memory_friendly]
corpus_tfidf.save('word2vecnih.tsv')
for doc in corpus_tfidf:
    print(doc)

In [ ]:
def normalize_text(text):

    text=text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(pic\.twitter\.com/[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    text = re.sub('#([^\s]+)', '', text)
    text = re.sub('[:;>?<=*+()&,\-#!$%\{˜|\}\[^_\\@\]1234567890’‘]',' ', text)
    text = re.sub('[\d]','', text)
    text = text.replace(".", '')
    text = text.replace("'", '')
    text = text.replace("`", '')
    text = text.replace("'s", '')
    text = text.replace("/", ' ')
    text = text.replace("\"", ' ')
    text = text.replace("\\", '')
    re.sub(' +', ' ', text)
    text=text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    #normalize some utf8 encoding
    text = text.replace("\x9d",'').replace("\x8c",'')
    text = text.replace("\xa0",'')
    text = text.replace("\x9d\x92", '').replace("\x9a\xaa\xf0\x9f\x94\xb5", '').replace("\xf0\x9f\x91\x8d\x87\xba\xf0\x9f\x87\xb8", '').replace("\x9f",'').replace("\x91\x8d",'')
    text = text.replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8",'').replace("\xf0",'').replace('\xf0x9f','').replace("\x9f\x91\x8d",'').replace("\x87\xba\x87\xb8",'')
    text = text.replace("\xe2\x80\x94",'').replace("\x9d\xa4",'').replace("\x96\x91",'').replace("\xe1\x91\xac\xc9\x8c\xce\x90\xc8\xbb\xef\xbb\x89\xd4\xbc\xef\xbb\x89\xc5\xa0\xc5\xa0\xc2\xb8",'')
    text = text.replace("\xe2\x80\x99s", "").replace("\xe2\x80\x98", '').replace("\xe2\x80\x99", '').replace("\xe2\x80\x9c", "").replace("\xe2\x80\x9d", "")
    text = text.replace("\xe2\x82\xac", "").replace("\xc2\xa3", "").replace("\xc2\xa0", "").replace("\xc2\xab", "").replace("\xf0\x9f\x94\xb4", "").replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8\xf0\x9f", "")
    text =  re.sub(r"\b[a-z]\b", "", text)
    text=re.sub( '\s+', ' ', text).strip()
    
    text=re.sub(r'\.+', ".", text)
    text=re.sub(r'\.\.+', ' ', text).replace('.', '')
    # Replace multiple dots with space
    text = re.sub('\.\.+', ' ', text) 
    # Remove single dots
    text = re.sub('\.', '', text)
    text = re.sub(r'\.{2,}', ' ', text)
    text = re.sub(r'\.{1}', '', text)
    
    return text

In [ ]:
df = pd.read_csv('abstract.csv', names=['info'])
df = df.astype(str)
sentences = df['info'].tolist()
normalized_sentences=[]
for sentence in sentences:
    norm_sent=normalize_text(sentence)
    normalized_sentences.append(norm_sent)
words = " ".join(normalized_sentences).split()
count = collections.Counter(words).most_common()
unique_words = [i[0] for i in count]
dic = {w: i for i, w in enumerate(unique_words)}
voc_size = len(dic)
data = [dic[word] for word in words]
cbow_pairs = []
for i in range(1, len(data)-1):
    cbow_pairs.append([[data[i-1], data[i+1]], data[i]]);
cbow_pairs_words = []
for i in range(1, len(words)-1):
    cbow_pairs_words.append([[words[i-1], words[i+1]], words[i]]);
skip_gram_pairs = []
for c in cbow_pairs:
    skip_gram_pairs.append([c[1], c[0][0]])
    skip_gram_pairs.append([c[1], c[0][1]])
def get_batch(size):
    assert size<len(skip_gram_pairs)
    X=[]
    Y=[]
    rdm = np.random.choice(range(len(skip_gram_pairs)),size, replace=False)
    for r in rdm:
        X.append(skip_gram_pairs[r][0])
        Y.append(skip_gram_pairs[r][1])     
    return X , Y
batch_size = 20
embedding_size = 2
num_sampled = 15
X = tf.placeholder(tf.int32, shape=[batch_size])
Y = tf.placeholder(tf.int32, shape = [batch_size, 1])
with tf.device("/cpu:0"):
    embeddings = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, X)
nce_weights = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))
nce_biases = tf.Variable(tf.zeros([voc_size]))
loss = tf.reduce_mean(tf.nn.nce_loss(nce_weights, nce_biases, Y, embed, num_sampled, voc_size))
optimizer = tf.train.AdamOptimizer(1e-1).minimize(loss)
epochs = 10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        batch_inputs, batch_labels = get_batch(batch_size)
        loss_val=sess.run([optimizer,loss], feed_dict = {X : batch_inputs, Y : batch_labels })
        if epoch % 1000 == 0:
            print("Loss at ", epoch, loss_val)
trained_embeddings = embeddings.eval()
trained_embeddings.save("word2vecnih.html")
#if trained_embeddings.shape[1] == 2:
   # labels = unique_words[:100]
    #for i, label in enumerate(labels):
        #x, y = trained_embeddings[i,:]
       # plt.scatter(x,y)
        #plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords = 'offset points', ha='right', va='bottom')
   # plt.savefig("word2vecnih.png")

**Downloading NSF Data**

In [ ]:
if not os.path.exists('./data'):
    os.makedirs('./data')
os.chdir('./data')
url = "https://www.nsf.gov/awardsearch/download.jsp"
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
links = soup.find_all("a", href = re.compile("download.*"))
links_correct = []
for link in links:
    links_correct.append('https://www.nsf.gov/awardsearch/' + link['href'])
for i, link in enumerate(links_correct[1:]):
    print("processing link {} of {}".format(i, len(links_correct[1:])))
    res = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(res.content))
    z.extractall()

In [ ]:
all_files = [file for file in os.listdir('.') if file.endswith(".xml")]

with open('../out.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow( ('file_name', 'directorate', 'division', 'title', 'institution', 'amount', 'grant type', 'abstract', 'date_start', 'date_end', 'program_officer', 'investigators', 'roles', 'number_pis') )
    for i, file in enumerate(all_files):
        try:
            file_name = file
            handler = open(file).read()
            soup = bs4.BeautifulSoup(handler, 'xml')

            directorate = soup.Directorate.LongName.text
            division = soup.Division.LongName.text
            title = soup.AwardTitle.text

            institution = soup.Institution.Name.text

            amount = soup.Award.AwardAmount.text
            grant_type = soup.Award.AwardInstrument.Value.text
            abstract = soup.Award.AbstractNarration.text

            date_end = soup.AwardExpirationDate.text
            date_start = soup.AwardEffectiveDate.text

            program_officer = soup.ProgramOfficer.text

            investigators = list()
            roles = list()
            for investigator in soup.find_all("Investigator"):
                investigators.append(investigator.FirstName.text + " " + investigator.LastName.text)
                roles.append(investigator.RoleCode.text)

            number_pis = len(investigators)

            try:
                writer.writerow( (file_name, directorate, division, title, institution, amount, grant_type, abstract, date_start, date_end, program_officer, investigators, roles, number_pis) )
            except:
                writer.writerow( ('NA', 'NA', 'NA', 'NA','NA','NA','NA','NA','NA','NA','NA','NA','NA') )
                print("problem writing the csv row")
        except:
            pass

        if i % 100 == 0:
            print("on the {}th file".format(i))
csvfile.close()

**Heat Map Distribution of Continuing NSF Awards**

In [ ]:
data = pd.read_csv('nsfheat.csv', names=['file_name', 'directorate', 'division', 'title', 'institution', 'amount', 'grant type', 'abstract', 'date_start', 'date_end', 'investigators', 'roles', 'number_pis', 'latitude', 'longitude'], low_memory=False)
f = folium.Map([20, 0], tiles= 'Mapbox Bright', zoom_start=2)
lat = data['latitude'].astype(float)
long = data['longitude'].astype(float)
heat_dat = data[['latitude', 'longitude']]
heat_dat = heat_dat.dropna(axis=0, subset=['latitude','longitude'])
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_dat.iterrows()]
HeatMap(heat_data).add_to(f)
f.save("heatmapnsf.html")

**Genderizer - Heat Map Distribution of Female PIs regarding Continuing NSF Awards**

In [ ]:
data = pd.read_csv('NSFF.csv', names=['latitude', 'longitude', 'name', 'gender'])
g = folium.Map([20, 0], tiles= 'Mapbox Bright', zoom_start=2)
lat = data['latitude'].astype(float)
long = data['longitude'].astype(float)
heat_dat = data[['latitude', 'longitude']]
heat_dat = heat_dat.dropna(axis=0, subset=['latitude','longitude'])
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_dat.iterrows()]
HeatMap(heat_data, gradient={.4: 'red', .65: 'green', 1: 'blue'}).add_to(g)
g.save("gheatmapnsf.html")

**Gensim**

In [2]:
# http://projector.tensorflow.org/ with 'abstractnsf.csv.txt' uploaded as metadata